In [17]:
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [9]:
reference_images = ImageFolder(root='../../datasets/armbench-object-id/Reference_Images/',)

In [10]:
pick_images_train = ImageFolder(root='../../datasets/armbench-object-id/Picks_splits/train/',)
pick_images_test = ImageFolder(root='../../datasets/armbench-object-id/Picks_splits/test/',)

In [2]:
import pandas as pd

obj = pd.read_pickle('../../datasets/armbench-object-id/train-test-split.pickle')

In [3]:
obj.keys()

dict_keys(['trainset', 'testset', 'trainset-objects', 'testset-objects'])

In [6]:
len(obj['testset'])

50000

In [9]:
import shutil
import os
for pick_id in obj['trainset']:
    shutil.move(f'../../datasets/armbench-object-id/Picks/{pick_id}', f'../../datasets/armbench-object-id/Picks_splits/train/{pick_id}')

In [10]:
for pick_id in obj['testset']:
    shutil.move(f'../../datasets/armbench-object-id/Picks/{pick_id}', f'../../datasets/armbench-object-id/Picks_splits/test/{pick_id}')

In [22]:
import json
label_dict = {}
for file in os.listdir('../../datasets/armbench-object-id/Picks_splits/train/'):
    with open(f'../../datasets/armbench-object-id/Picks_splits/train/{file}/annotation.json', 'r') as f:
        annotation = json.load(f)
    label_dict[file] = annotation["GT_ID"]

# save into json file
with open('../../datasets/armbench-object-id/Picks_splits/train_label_dict.json', 'w') as f:
    json.dump(label_dict, f)

In [23]:
label_dict = {}
for file in os.listdir('../../datasets/armbench-object-id/Picks_splits/test/'):
    with open(f'../../datasets/armbench-object-id/Picks_splits/test/{file}/annotation.json', 'r') as f:
        annotation = json.load(f)
    label_dict[file] = annotation["GT_ID"]

with open('../../datasets/armbench-object-id/Picks_splits/test_label_dict.json', 'w') as f:
    json.dump(label_dict, f)


In [ ]:
import json
with open('../../datasets/armbench-object-id/Picks_splits/train_label_dict.json', 'r') as f:
    train_labels = json.load(f)
train_labels[0:10]

In [24]:
import json
with open('../../datasets/armbench-object-id/Picks_splits/test_label_dict.json', 'r') as f:
    test_labels = json.load(f)

In [18]:
no_ref_pick_ids = []
valid_train_labels = train_labels.copy()
print("Before cleaning: ", len(valid_train_labels), " classes")
for key, item in tqdm(train_labels.items()):
    if item not in reference_images.classes:
        valid_train_labels.pop(key)
        print(key, item)
        no_ref_pick_ids.append(key)
print("After cleaning: ", len(valid_train_labels), " classes")

Before cleaning:  185495  classes


 63%|██████▎   | 117558/185495 [01:03<00:36, 1880.52it/s]

008023001652744517593 0307975878


 83%|████████▎ | 153047/185495 [01:22<00:18, 1800.23it/s]

011056001651754233153 B074DXQ4V9


100%|██████████| 185495/185495 [01:38<00:00, 1879.09it/s]

After cleaning:  185493  classes


In [25]:
no_ref_pick_ids = []
valid_test_labels = test_labels.copy()
print("Before cleaning: ", len(valid_test_labels), " classes")
for key, item in tqdm(test_labels.items()):
    if item not in reference_images.classes:
        valid_test_labels.pop(key)
        print(key, item)
        no_ref_pick_ids.append(key)
print("After cleaning: ", len(valid_test_labels), " classes")

Before cleaning:  50000  classes


100%|██████████| 50000/50000 [00:26<00:00, 1855.89it/s]

After cleaning:  50000  classes


In [21]:
import shutil
import os
shutil.rmtree('../../datasets/armbench-object-id/Picks_splits/train/008023001652744517593')

In [22]:
import shutil
import os
shutil.rmtree('../../datasets/armbench-object-id/Picks_splits/train/011056001651754233153')

In [19]:
with open('../../datasets/armbench-object-id/Picks_splits/train_label_dict.json', 'w') as f:
    json.dump(valid_train_labels, f)

with open('../../datasets/armbench-object-id/Picks_splits/no_ref_pick_ids.json', 'w') as f:
    json.dump(no_ref_pick_ids, f)

In [20]:
'X00308OLCX' in valid_train_labels.keys()


False

In [82]:
pick_images.class_to_idx['001001001652120609820']

3

In [74]:
type(pick_images.targets)

list

In [76]:
import numpy as np
np.array(pick_images.targets)==0

array([ True,  True,  True, ..., False, False, False])

In [83]:
[idx for idx, x in enumerate(pick_images.targets) if x == 3]

[10, 11, 12, 13]

# build clean dataset

In [31]:
from tqdm import tqdm
import shutil
pick_up_classes = pick_images_train.classes.copy()
print("Before cleaning: ", len(pick_up_classes), " classes")
target_ids = []
for clsidx in tqdm(range(len(pick_images_train.classes))):
    ids = [idx for idx, x in enumerate(pick_images_train.targets) if x == clsidx]
    if len(ids) < 3:
        pick_up_classes.remove(pick_images_train.classes[clsidx])
        continue
    if len(ids) > 3:
        ids = ids[1:]
    shutil.move(f"../../datasets/armbench-object-id/Picks_splits/train/{pick_images_train.classes[clsidx]}/", f"../../datasets/armbench-object-id/Picks_splits/train_over3image/{pick_images_train.classes[clsidx]}/")
    target_ids.append(ids)

print("After cleaning: ", len(target_ids), " classes")
np.save('../../datasets/armbench-object-id/Picks_splits/train_target_ids_over3image.npy', target_ids)
np.save('../../datasets/armbench-object-id/Picks_splits/train_classes_over3image.npy', pick_up_classes)

Before cleaning:  185495  classes


 63%|██████▎   | 117160/185495 [24:40<14:23, 79.13it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/armbench-object-id/Picks_splits/train/008023001652744517593/'